In [38]:
import pandas as pd
import numpy as np

 # 01_ocean

In [75]:
N1, M2 = map(int, input().split())
data = pd.DataFrame([[val for val in map(int, input().split())] for row in range(N1)]) # And where is Parquet???
data

,0,1,2,3,4,5
0,1,4,4,2,1,1
1,0,0,1,0,-1,-1
2,-2,-3,-1,-5,-6,-4
3,-1,-3,-3,-4,-4,-2


In [81]:
def deep_area(data: pd.DataFrame):
    '''
    I think np.nansum() faster than .sum().sum() and also more readable than .to_numpy().sum() or .values.sum()
    '''
    return np.nansum((data < -5), dtype=np.intc)

def water_volume(data: pd.DataFrame):
    return np.nansum(data[data < 0].abs(), dtype=np.intc)

def max_height(data: pd.DataFrame):
    return int(np.nanmax(data[data >= 0]))



In [88]:
print(deep_area(data), water_volume(data), max_height(data), sep='\n')

1
40
4


# 03_gamestats

In [349]:
games = pd.read_csv('./data_for_17/games001.csv', sep=';', index_col=0) # Why don't we study Parquet? 
rates = pd.read_csv('./data_for_17/rates001.csv', sep=';', index_col=0) # I think Parquet as useful as Pickle...

In [350]:
mean_marks = rates.groupby(by='id').mean()
games['mean_mark'] = mean_marks
games

,name,company,mean_mark
id,,,
1,XCOM 2,Firaxis,8.866667
2,Civilization V,Firaxis,8.333333
3,Darkest Dungeon,Red Hook Studios,9.000000
4,Star Traders,Trese Brothers,8.000000


In [212]:
top3_games = games.sort_values(by='mean_mark', ascending=False).drop(columns='company', inplace=False).iloc[:3]

# I couln't do anything better...
output = top3_games.round(3).to_string(index=False, header=False).split('\n')
for row in range(3):
    print(output[row].lstrip())

Darkest Dungeon 9.000
XCOM 2 8.867
Civilization V 8.333


In [234]:
top_game_counts = games[games['mean_mark'] > 8].groupby('company').count()['name']
top_company = top_game_counts.idxmax()
print(top_company, top_game_counts[top_company])

Firaxis 2


# 05_gol

In [340]:
N2, M2, K2 = map(int, input().split())
ground_list = [[val for val in map(lambda x: 0 if x == '.' else 1, [*input()])] for row in range(N2)]
ground = pd.DataFrame(ground_list)
ground

,0,1,2,3,4,5,6,7,8,9
0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0
2,0,1,1,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0


In [341]:
class for_match:
    last_row = N2 - 1
    last_col = M2 - 1

In [342]:
def update(ground, K):
    if K == 0:
        return ground
    
    new_ground = pd.DataFrame(np.zeros((N2, M2)), dtype=int)
    
    for row in range(N2):
        for col in range(M2):
            current_cell = ground.iloc[row, col]
            match row, col:
                case 0, 0:
                    num_alive = ground.iloc[:2, :2].to_numpy().sum() - current_cell
                case for_match.last_row, 0:
                    num_alive = ground.iloc[-2:, :2].to_numpy().sum() - current_cell
                case _, 0:
                    num_alive = ground.iloc[row-1:row+2, :2].to_numpy().sum() - current_cell
                case 0, for_match.last_col:
                    num_alive = ground.iloc[:2, -2:].to_numpy().sum() - current_cell
                case 0, _:
                    num_alive = ground.iloc[:2, col-1:col+2].to_numpy().sum() - current_cell
                case _, _:
                    num_alive = ground.iloc[row-1:row+2, col-1:col+2].to_numpy().sum() - current_cell
                
            if current_cell == 1 and (num_alive == 2 or num_alive == 3):
                new_ground.iloc[row, col] = 1
            elif current_cell == 0 and num_alive == 3:
                new_ground.iloc[row, col] = 1
    
    ground.iloc[:, :] = new_ground

    return update(ground, K-1)

In [343]:
update(ground, K2)

,0,1,2,3,4,5,6,7,8,9
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0
5,0,0,0,0,1,1,1,0,0,0


In [344]:
print(ground.replace([0, 1], ['.', '#']).to_string(index=False, header=False).replace(' ', ''))

..........
..........
..........
.....#....
......#...
....###...
